In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/train (2).csv")

# Display the first few rows of the dataset to verify it was loaded correctly
df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('punkt')

# Function to preprocess text
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Convert tokens to lowercase
    tokens = [word.lower() for word in tokens]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Apply preprocessing to each comment in the dataset
df['cleaned_comment'] = df['comment_text'].apply(preprocess_text)

# Display the preprocessed dataset
df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_comment
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[daww, matches, background, colour, im, seemin..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, im, really, trying, edit, war, guy,..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[cant, make, real, suggestions, improvement, w..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page, thats]"


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['comment_text'])
X = tokenizer.texts_to_sequences(df['comment_text'])

# Pad sequences to ensure uniform length
max_len = 100  # You can adjust max_len as needed
X = pad_sequences(X, maxlen=max_len)

# Define the target variable
y = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define the neural network architecture with Bidirectional LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(6, activation='sigmoid')
])

# Compile the model
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=64, validation_split=0.2)

# Save the model architecture as JSON
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights
model.save_weights("model_weights.h5")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)




Epoch 1/2
1596/1596 [==============================] - 2345s 1s/step - loss: 0.0800 - accuracy: 0.9657 - val_loss: 0.0514 - val_accuracy: 0.9943
Epoch 2/2
998/998 [==============================] - 160s 160ms/step - loss: 0.0503 - accuracy: 0.9941
Test Accuracy: 0.9941093325614929


In [13]:
import pickle

# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)




In [6]:
# from tensorflow.keras.models import load_model

# # Load the model
# model = load_model('/content/drive/MyDrive/toxicity_model.h5')


In [ ]:
# # Example text
# example_text = "he is an jerk "

# # Preprocess the example text
# example_sequence = tokenizer.texts_to_sequences([example_text])
# example_padded_sequence = pad_sequences(example_sequence, maxlen=max_len)

# # Make predictions
# predictions = model.predict(example_padded_sequence)

# # Display predictions
# # Convert probabilities to percentages
# toxicity_labels = ['Toxic', 'Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate']
# probabilities_percentages = [f"{label}: {prob * 100:.2f}%" for label, prob in zip(toxicity_labels, predictions[0])]

# # Display probabilities
# for probability in probabilities_percentages:
#     print(probability)



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/train (2).csv")

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['comment_text'])
X = tokenizer.texts_to_sequences(df['comment_text'])
max_len = 100  # Same as defined during training
X = pad_sequences(X, maxlen=max_len)

# Define the target variable
y = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

# Load the model architecture
with open('model_architecture.json', 'r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# Load model weights
loaded_model.load_weights("model_weights.h5")

# Compile the model (if not compiled before saving)
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [10]:
# Example text
example_text = "i will kill you "

# Preprocess the example text
example_sequence = tokenizer.texts_to_sequences([example_text])
example_padded_sequence = pad_sequences(example_sequence, maxlen=max_len)

# Make predictions
predictions = loaded_model.predict(example_padded_sequence)

# Display predictions
toxicity_labels = ['Toxic', 'Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate']
probabilities_percentages = [f"{label}: {prob * 100:.2f}%" for label, prob in zip(toxicity_labels, predictions[0])]
for probability in probabilities_percentages:
    print(probability)


1/1 [==============================] - 0s 88ms/step
Toxic: 83.41%
Severe Toxic: 1.23%
Obscene: 21.39%
Threat: 2.49%
Insult: 32.55%
Identity Hate: 4.53%
